https://en.seedfinder.eu/database/breeder/

Idea: scrapear y obtener los datos de todas las geneticas

Idea 2: generar vectores con los datos de cada genetica para calcular los N mas similares a una consulta. 

In [43]:
# Installs
import pandas as pd
import numpy as np
import os
import re
import time
import random
from time import sleep
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True
!pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager

#Spacy
!python -m spacy download en
!pip install spacy
import spacy
from spacy import displacy

# get_names
!pip install thefuzz[speedup]
from thefuzz import process, fuzz

     --------------------------------------- 12.8/12.8 MB 12.1 MB/s eta 0:00:00
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


     ---------------------------------------- 50.5/50.5 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for python-levenshtein
Failed to build python-levenshtein
  Running setup.py install for python-levenshtein: started
  Running setup.py install for python-levenshtein: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [28 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.9
  creating build\lib.win-amd64-3.9\Levenshtein
  copying Levenshtein\StringMatcher.py -> build\lib.win-amd64-3.9\Levenshtein
  copying Levenshtein\__init__.py -> build\lib.win-amd64-3.9\Levenshtein
  running egg_info
  writing python_Levenshtein.egg-info\PKG-INFO
  writing dependency_links to python_Levenshtein.egg-info\dependency_links.txt
  deleting python_Levenshtein.egg-info\entry_points.txt
  writing namespace_packages to python_Levenshtein.egg-info\namespace_packages.txt
  writing requirements to python_Levenshtein.egg-info\requires.txt
  writing top-level names to python_Levenshtein.egg-info\top_level.txt
  reading manifest file 'python_Levenshtein.egg-info\SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license file 'CO

In [7]:
def get_links(navegacion_con_headers = False):
    os.chdir('C:/Users/Felipe/Desktop/Proyectos/Weed')
    
    # instanciamos webdriver (True or False)
    if navegacion_con_headers == True:
        driver = webdriver.Chrome(ChromeDriverManager().install())
    else:
        driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    
    # ir a la pagina con bancos de semillas
    driver.get('https://en.seedfinder.eu/database/breeder/')
    
    # Link a todos los bancos de semillas
    links_bancos = driver.find_elements('xpath','//*[@id="cannabis-breeders-table"]/li/span[2]/a')
    links_bancos = [elem.get_attribute('href') for elem in links_bancos]
    
    #Iterar por banco
    links_geneticas = []
    for i,link in enumerate(links_bancos):
        driver.get(link)
        sleep(5)
        geneticas_del_banco = driver.find_elements('xpath','//*[@id="TABLE_1"]/tbody/tr/th/a')
        geneticas_del_banco = [elem.get_attribute('href') for elem in geneticas_del_banco]
        links_geneticas.extend(geneticas_del_banco)
        print(f'\n\nIteracion {i} de {len(links_bancos)}\n{len(geneticas_del_banco)} geneticas encontradas en esta iteracion ({len(links_geneticas)}) en total')
        
        # Guardado de links de geneticas
        results = open('links.txt','w')
        for i in links_geneticas:
            results.write(str(i))
            results.write('\n')
        results.close()
        
    # leer links
    with open('links.txt') as f:
        links = f.readlines()

    return [l.replace('\n','') for l in links]

In [77]:
def get_data(navegacion_con_headers = False):

    # instanciamos webdriver (True or False)
    if navegacion_con_headers == True:
        driver = webdriver.Chrome(ChromeDriverManager().install())
    else:
        driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

    #read links
    os.chdir('C:/Users/Felipe/Desktop/Proyectos/Weed')
    with open('links.txt') as f:
        links = f.readlines()
    links = [l.replace('\n','') for l in links]

    #Iterate links and scrape
    names = []
    info = []
    images = []

    for i, link in enumerate(links): 
        try:
            driver.get(link)
            sleep(1.5)
            print(f'\n\n{i} de {len(links)}:')

            # Nombre
            try:
                name = driver.find_element('xpath','//*/div/h1').text
            except:
                name = ''

            # Info
            try:
                basic1 = driver.find_element_by_xpath('//*[@id="bscinfo"]').text
            except:
                basic1 = ''
            try:
                basic2 = driver.find_element_by_xpath('//*[@id="prces"]/div[2]').text
            except:
                basic2 = ''

            final = basic1 + basic2


            # Imagen
            try:
                imagen = driver.find_element_by_xpath('//*[@id="bscinfo"]/div[2]/p[2]/span/span/span/a/img').get_attribute('src')
            except:
                try:
                    imagen = driver.find_element_by_xpath('//*[@id="dapics"]/div[2]/div/span/a/picture/img').get_attribute('src')
                except:
                    imagen = ''

            names.append(name)
            print(f'Name: {[True if len(names[-1])>1 else False]}, len: {len(names)}')

            info.append(final)
            print(f'Info: {[True if len(info[-1])>1 else False]}, len: {len(info)}')

            images.append(imagen)
            print(f'Imagen: {[True if len(images[-1])>1 else False]}, len: {len(images)}')

            if len(names) != len(info) or len(names) != len(images):
                print('Stop: lists length dont match')
                break

        except:
            pass

    # Dir with data
    try:
        os.mkdir('data')
    except:
        pass
    os.chdir('data')
    
    
    if save_txt == True:
        # Save
        results = open('names.txt','w', encoding="utf-8")
        for i in names:
            results.write(str(i))
            results.write('\n')
        results.close()

        # Save
        results = open('info.txt','w', encoding="utf-8")
        for i in info:
            results.write(str(i))
            results.write('\n')
        results.close()

        # Save
        results = open('links.txt','w', encoding="utf-8")
        for i in links:
            results.write(str(i))
            results.write('\n')
        results.close()

        # Save
        results = open('images.txt','w', encoding="utf-8")
        for i in images:
            results.write(str(i))
            results.write('\n')
        results.close()


    df = pd.DataFrame(list(zip(names,info,links,images)), columns =['names','info','links','images'])

    # Descarga
    df.to_csv('results.csv',index=False)
    
    return df

In [2]:
# READ CSV
os.chdir('C:/Users/Felipe/Desktop/Proyectos/Weed/Data3')
df = pd.read_csv('results.csv')
for col in df.columns:
    df[col] = df[col].astype(str)
    
#Info into list
info = list(df['info'].values)

In [4]:
i = random.randint(0, len(info))
print(info[i])

Basic / Breeders Info
Cataract Kush is an indica variety from DNA Genetics and can be cultivated indoors (where the plants will need a flowering time of ±60 days) and outdoors. DNA Genetics' Cataract Kush is a THC dominant variety and is/was also available as feminized seeds.
DNA Genetics' Cataract Kush Description
This is the combination of two known winners, LA Confidential and OG Kush.

She finishes in 8-9 weeks and has blankets of crystals! The Cataract Kush looks almost grey-black when cured.
The flavor is LA dominate with the OG Kush coming thru in the exhale. The nugs look OG with LA density.
This is the perfect combination of these two strains and the high is VERY strong!

The effect is good for pain, eating and sleep disorders. Most describe Catract as a "creeper" high that keeps on building long after you stop smoking!
This strain has been years in the making and is NOT recomended for the light weight smoker!!

Above average yields and dense frosty nugs are easily achieved by

In [45]:
def get_name(name):
    lista_names = list(df['names'].values)
    if name in lista_names:
        return name
    
    mejor_match, puntaje = process.extractBests(name.strip(), lista_names, scorer=fuzz.token_set_ratio)[0]
    return print(f'No se encontro el nombre, tal vez quiciste decir: {mejor_match}')

In [67]:
def get_similars(name):

    name = 'Kush Kush'
    top_n = 3

    info_cleaned = [i.replace('\n','') for i in info]

    nlp = spacy.load('en_core_web_sm')

    indice_interes = list(df['names'].values).index(name)

    doc_query = nlp(info_cleaned[indice_interes])

    best_sim = [0]*top_n
    indices = []

    for i,inf in enumerate(info_cleaned):

        doc_key = nlp(inf)
        sim = doc_query.similarity(doc_key)

        for t in range(top_n):
            if sim > best_sim[t] and sim < 1:
                best_sim[t] = sim
                indices.append(i)

    print(f'QUERY INFO:\n {name} \n {df["links"][indice_interes]} \n {df["info"][indice_interes]} \n')
    print(f'TOP {top_n} FOUND INFO:')
    for i,x in enumerate(indices):
        print(f'----------- NUMBER {i} ----------- ')
        print(df['names'][x], '\n')
        print(df['info'][x], '\n')
        print(df['links'][x], '\n')
        print(f'SIMILARITY: {best_sim[i]} \n\n')

In [48]:
get_name('Kush Magic')

No se encontro el nombre, tal vez quiciste decir: Kush Kush


In [ ]:
get_similars('Kush Kush')

C:\Users\Felipe\AppData\Local\Temp\ipykernel_18540\3462037353.py:20: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  sim = doc_query.similarity(doc_key)
